In [220]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from geopy.geocoders import Nominatim
from tqdm import tqdm
from geopy.distance import geodesic
import folium
from folium.plugins import MarkerCluster
import math
import datetime
import geopandas as gpd
import urllib.request
import requests
import json
import openmeteo_requests
import requests_cache
from shapely.geometry import Polygon, Point
from retry_requests import retry
from shapely.wkt import loads
import random 
from pathlib import Path

In [231]:
DATA_DIR_GEBOUWEN = Path("src/data_gebouwen")
DATA_DIR_BOMEN = Path("src/data_bomen")
BOUWJAAR_DATA_PATH = DATA_DIR_GEBOUWEN / 'BOUWJAAR.csv'
INCIDENT_DATA_PATH = DATA_DIR_GEBOUWEN / 'Building_incident_with_weather_data.csv'



C:\Users\Aliha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,Incident_ID,Date,Incident_Starttime,Hour,Date_time,Incident_Endtime,Incident_Duration,Incident_Priority,Service_Area,Municipality,...,wind_direction_100m,wind_gusts_10m,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm
1563,409455,2022-09-03,19:49:48,19,2022/09/03 19:49,20:47:07,00:57:19,2.0,Willem,Amsterdam,...,76.908104,12.240000,21.472000,18.922001,17.272001,13.072000,0.446,0.460,0.459,0.621
1564,410122,2022-09-12,20:16:49,20,2022/09/12 20:16,20:56:03,00:39:14,1.0,Nico,Amsterdam,...,249.676773,9.000000,19.061001,17.911001,17.211000,13.461000,0.526,0.460,0.441,0.612
1565,410423,2022-09-16,15:41:52,15,2022/09/16 15:41,17:45:48,02:03:56,2.0,IJsbrand,Amsterdam,...,316.123199,50.039997,16.517500,15.817500,16.867500,13.517500,0.529,0.473,0.448,0.615
1566,410427,2022-09-16,16:23:41,16,2022/09/16 16:23,17:12:35,00:48:54,2.0,Hendrik,Amsterdam,...,324.706573,54.719997,15.828500,15.778501,16.828499,13.478500,0.570,0.503,0.461,0.616
1567,410474,2022-09-17,14:18:13,14,2022/09/17 14:18,15:00:41,00:42:28,2.0,Dirk,Amsterdam,...,324.806000,52.560001,14.754500,14.704500,16.704500,13.554501,0.594,0.508,0.462,0.616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1731,443438,2023-11-02,16:53:00,16,2023/11/02 16:53,16:53:58,00:00:58,2.0,IJsbrand,Amsterdam,...,193.686478,80.639999,9.761001,11.061001,12.661000,13.911000,0.756,0.755,0.733,0.650
1732,443441,2023-11-02,16:56:39,16,2023/11/02 16:56,16:57:26,00:00:47,2.0,Zebra,Amsterdam,...,192.528793,86.760002,10.167500,11.317500,12.867499,14.067500,0.741,0.729,0.683,0.622
1733,443453,2023-11-02,17:21:45,17,2023/11/02 17:21,17:22:39,00:00:54,2.0,Willem,Amsterdam,...,196.389618,81.720001,9.654500,11.204500,12.804501,13.954500,0.746,0.745,0.708,0.631
1734,443456,2023-11-02,17:31:11,17,2023/11/02 17:31,17:31:42,00:00:31,2.0,Victor,Amsterdam,...,196.023041,83.159996,9.924000,11.224000,12.774000,13.974000,0.736,0.727,0.680,0.621


In [233]:
DATA_DIR_BOMEN = Path("src/data_bomen")
DATA_DIR_GEBOUWEN = Path("src/data_gebouwen")

INCIDENT_DATA_PATH = DATA_DIR_BOMEN / 'Incidenten_oorspronkelijk_volledig.csv'
BOUWJAAR_DATA_PATH = DATA_DIR_GEBOUWEN / 'BOUWJAAR.csv'
ZIPCODE_JSON_PATH = DATA_DIR_BOMEN / "zipcodes_boxes.json"
GRID_SIZE = 200     ## GRID SIZE IN METERS

BUILDING_DATA_CLEAN_PATH = DATA_DIR_GEBOUWEN / f"building_geo_data_clean_{str(GRID_SIZE)}.csv"
GRID_DATA_PATH = DATA_DIR_GEBOUWEN / f"grid_enriched_buildings_{GRID_SIZE}.csv"
INCIDENTS_WEATHER_PATH = DATA_DIR_GEBOUWEN / "Building_incident_with_weather_data.csv"
INCIDENTS_WEATHER_GEO_PATH = DATA_DIR_GEBOUWEN / f"incidents_weather_geo_buildings_{GRID_SIZE}.csv"

POSITIVE_SAMPLES_PATH = DATA_DIR_GEBOUWEN / f"positive_samples_buildings_{GRID_SIZE}.csv"
NEGATIVE_SAMPLES_PATH = DATA_DIR_GEBOUWEN / f"negative_samples_buildings_{GRID_SIZE}.csv"

ZIP_KEY = "Zipcode"
ZIP4_KEY = "Zip4"

DATE_WINDOW = 7

AMSTERDAM_BBOX = (52.26618, 4.64663, 52.475115999999994, 5.150491999999999)

In [234]:
BUILDING_COLUMNS = [
    "OBJECTNUMMER",
    "grid_id",
    "Bouwjaar",       
    "WKT_LNG_LAT",
    "WKT_LAT_LNG",
    "LNG",
    "LAT"
]

SERVICE_AREAS_OUT_OF_SCOPE = [
    "Amstelveen",
    "Aalsmeer",
    "Uithoorn"
]

RF_INCIDENT_COLUMNS = [
    "Incident_ID",
    "Service_Area",
    "grid_id",
    "Date",
    "Hour",
    "temperature_2m",
    "relative_humidity_2m",
    "dew_point_2m",
    "apparent_temperature",
    "precipitation",
    "rain",
    "snowfall",
    "snow_depth",
    "weather_code",
    "pressure_msl",
    "surface_pressure",
    "wind_speed_10m",
    "wind_direction_10m",
    "wind_gusts_10m",
    "soil_temperature_0_to_7cm",
    "soil_temperature_7_to_28cm",
    "soil_temperature_28_to_100cm",
    "soil_temperature_100_to_255cm",
    "soil_moisture_0_to_7cm",
    "soil_moisture_7_to_28cm",
    "soil_moisture_28_to_100cm",
    "soil_moisture_100_to_255cm",
]

RF_GRID_COLUMNS = [
    "grid_id",
    "has_building",
    "Bouwjaar",
]

RF_BUILDING_COLUMNS = [
    "OBJECTNUMMER",
    "Bouwjaar",       
    "WKT_LNG_LAT",
    "WKT_LAT_LNG",
    "LNG",
    "LAT"
]

In [235]:
def create_grid_gdf():
    #geolocator = Nominatim(user_agent="user_agent")

    # Get coordinates for Amsterdam
    #location = geolocator.geocode("Amsterdam, Netherlands")
    # amsterdam_lat, amsterdam_lon = location.latitude, location.longitude
    amsterdam_lat, amsterdam_lon = [4.88, 52.32]

    amsterdam_bbox = AMSTERDAM_BBOX

    # Calculate grid bounds
    lat_step = GRID_SIZE / 111000  # 1 degree of latitude is approximately 111 kilometers
    lon_step = (GRID_SIZE / 111000) / np.cos(np.radians(amsterdam_lat))  # Correct for latitude

    grid_polygons = []
    for lat in np.arange(amsterdam_bbox[0], amsterdam_bbox[2], lat_step):
        for lon in np.arange(amsterdam_bbox[1], amsterdam_bbox[3], lon_step):
            polygon = Polygon([
                (lon, lat),
                (lon + lon_step, lat),
                (lon + lon_step, lat + lat_step),
                (lon, lat + lat_step),
                (lon, lat),
            ])
            grid_polygons.append(polygon)

    grid_gdf = gpd.GeoDataFrame(geometry=grid_polygons, crs="EPSG:4326")
    return grid_gdf

In [236]:
def create_building_incident_gdf(incidents_weather_df, buildings, grid_gdf):
    # Filter on areas in scope
    incidents_weather_df = incidents_weather_df[~incidents_weather_df.Service_Area.isin(SERVICE_AREAS_OUT_OF_SCOPE)]

    # create gdf from buildings
    building_gdf = gpd.GeoDataFrame(buildings, geometry=gpd.points_from_xy(buildings['LNG'], buildings['LAT']), crs="EPSG:4326")
    # create gdf from indicents
    incident_gdf = gpd.GeoDataFrame(incidents_weather_df, geometry=gpd.points_from_xy(incidents_weather_df['LON'], incidents_weather_df['LAT']), crs="EPSG:4326")
    #join with grid gdf
    building_gdf = gpd.sjoin(building_gdf, grid_gdf, how="left", op="within")
    incident_gdf = gpd.sjoin(incident_gdf, grid_gdf, how="left", op="within")

    #clean up gdf
    building_gdf = building_gdf.rename(columns={"index_right" : "grid_id", "geometry" : "location"})
    incident_gdf = incident_gdf.rename(columns={"index_right" : "grid_id", "geometry" : "location"})

    return building_gdf, incident_gdf

In [237]:
def convert_cat_to_avg(
    cat_values,
    delimeter = "-"
):
    ''' 
    Converts categorical values to means of type float
    Splits cat values on delimter, computes the mean for each cat
    Returns mean of all means of the categories
    '''
    means = []
    for cat in cat_values:
        if not isinstance(cat, str):
            continue
        if not delimeter in cat:
            continue
        vals = cat.split(delimeter)
        means.append(np.mean([float(val) for val in vals]))
    m = round(np.mean(means), 3)
    return 0 if np.isnan(m) else m

In [238]:
def enrich_grid_df(
    grid_gdf,
    building_gdf
):
    for i in grid_gdf.index:
        building_sub_df = building_gdf[building_gdf.grid_id == i]
        if len(building_sub_df)>0:
            # Compute and add averages for height, diameter and year
            grid_gdf.at[i, "Bouwjaar"] = round(np.mean(building_sub_df.Bouwjaar.values), 3)
            # Add soortnaam counts
            # for name, count in building_sub_df.soortnaamKort.value_counts().items():
            #     grid_gdf.at[i, "has_building"] = True
            #     grid_gdf.at[i, name] = count
        else:
            grid_gdf.at[i, "has_building"] = False

    return grid_gdf, building_gdf

In [239]:
def save_data(building_gdf, incident_gdf, grid_gdf):
    # save data
    building_gdf.to_csv(BUILDING_DATA_CLEAN_PATH, sep=",", encoding="utf-8", index=False)
    incident_gdf.to_csv(INCIDENTS_WEATHER_GEO_PATH, sep=",", encoding="utf-8", index=False)

    # clean and save data
    grid_gdf = grid_gdf.fillna(0)
    grid_gdf[grid_gdf.has_building == True]
    grid_gdf['grid_id'] = grid_gdf.index
    grid_gdf.to_csv(GRID_DATA_PATH, sep=",", encoding="utf-8", index=False)


def create_save_positives(incident_gdf, building_gdf, grid_gdf):
    incident_gdf.Date = pd.to_datetime(incident_gdf.Date)
    # Pick necessary columns
    incident_sub_gdf = incident_gdf[RF_INCIDENT_COLUMNS]

    grid_gdf['grid_id'] = grid_gdf.index #?
    grid_sub_gdf = grid_gdf[RF_GRID_COLUMNS]

    building_gdf = building_gdf.rename(columns={"id" : "building_id"})
    building_sub_gdf = building_gdf[RF_BUILDING_COLUMNS]
    positive_samples = grid_sub_gdf.merge(incident_sub_gdf, on='grid_id', how='inner')
    positive_samples.to_csv(POSITIVE_SAMPLES_PATH, sep=",", encoding="utf-8", index=False)
    return positive_samples

In [240]:
def verify_sample(
    incidents,
    grid_id,
    date,
    window = DATE_WINDOW
):
    start_date = date - pd.DateOffset(days=window)
    end_date = date + pd.DateOffset(days=window)

    grids = incidents[(incidents['Date'] >= start_date) & (incidents['Date'] <= end_date)].values

    return False if grid_id not in grids else True


def create_save_negatives(
    positives,
    incidents,
    grid
):
    grids_with_building = list(grid[grid.has_building == True].grid_id.values)
    negatives = positives[['Date', 'Hour']]
    negatives[RF_GRID_COLUMNS] = None

    for i, row in negatives.iterrows():
        random_grid = random.sample(grids_with_building, 1)[0]
        while(verify_sample(incidents, random_grid, row.Date)):
            random_grid = random.sample(grids_with_building, 1)[0]
        grid_data = grid[grid.grid_id == random_grid][RF_GRID_COLUMNS].reset_index(drop=True)
        negatives.loc[i, RF_GRID_COLUMNS] = grid_data.iloc[0]
    # save
    negatives.to_csv(NEGATIVE_SAMPLES_PATH, sep=",", encoding="utf-8", index=False)
    return negatives

In [243]:
from src.GetWeather import GetWeather

incident_df = pd.read_csv(INCIDENT_DATA_PATH, sep=",", encoding="utf-8")
#incident_df = incident_df.drop(['Unnamed: 0'], axis=1)
incident_df = incident_df.set_index('Incident_ID')

# create datasets
building_df = pd.read_csv(BOUWJAAR_DATA_PATH, sep=";", encoding="utf-8")
building_df['WKT_LNG_LAT'] = building_df['WKT_LNG_LAT'].apply(loads)
building_df['WKT_LAT_LNG'] = building_df['WKT_LAT_LNG'].apply(loads)

incidents_weather_df = pd.read_csv(INCIDENTS_WEATHER_PATH, sep=",", encoding="utf-8")
grid_gdf = create_grid_gdf()

incidents_weather_df = incidents_weather_df[~incidents_weather_df.Service_Area.isin(SERVICE_AREAS_OUT_OF_SCOPE)]
df_tree_incidents = incident_df[incident_df["Damage_Type"]=="Building"]
building_gdf, incident_gdf = create_building_incident_gdf(incidents_weather_df=incidents_weather_df, buildings=building_df, grid_gdf=grid_gdf)

# get rid of unnecessary columns
building_gdf = building_gdf[BUILDING_COLUMNS]
grid_gdf, building_gdf = enrich_grid_df(grid_gdf=grid_gdf, building_gdf=building_gdf)

# save data
save_data(building_gdf=building_gdf, incident_gdf=incident_gdf, grid_gdf=grid_gdf)

positive_samples = create_save_positives(incident_gdf=incident_gdf, building_gdf=building_gdf, grid_gdf=grid_gdf)
negative_samples = create_save_negatives(positives=positive_samples, incidents=incident_gdf, grid=grid_gdf)

weather_getter = GetWeather(grid_path=GRID_DATA_PATH, samples_path=NEGATIVE_SAMPLES_PATH, sleep_time=90)  
negative_samples = weather_getter.add_weather_data()


C:\Users\Aliha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3526: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Aliha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3526: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Aliha\AppData\Local\Temp\ipykernel_11456\575692110.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dt

ValueError: Sample larger than population or is negative